# Test/Dev Neural Network Architecture

In [ ]:
from __future__ import print_function

import sys
import os
import time
import chb

import numpy as np
import theano
import theano.tensor as T

import lasagne

## Load Subject Data

In [ ]:
# TODO: organize this differently - this should just load the subject and
#       data, so that we can loop through all the LOOs in main
# Load data for subject
subject = chb.CHBsubj()
subject.load_meta('chb03')
subject.load_data(exthd=False)

In [ ]:
# Load and read training and test set images and labels.
x_train, y_train, x_test, y_test = chb.leaveOneOut(subject, 1, 1000, 100)

# We reserve the last 100 training examples for validation.
x_train, x_val = x_train[:-100], x_train[-100:]
y_train, y_val = y_train[:-100], y_train[-100:]

----

## CNN from Scratch
Well, not entirely. It's based on notes found [here](http://cs231n.github.io/neural-networks-1/) 

In [ ]:
data_size = (None, 1, 23, 256)
output_size = 1 # Not sure if this is right! Maybe it should be 2?

In [ ]:
from lasagne import layers
from lasagne.nonlinearities import rectify, leaky_rectify, sigmoid
from lasagne.objectives import binary_crossentropy

The standard weight initialization for layers below is `W=lasagne.init.GlorotUniform()`, which we will not change (for now). It may be useful to change this to `lasagne.init.He()` or `lasagne.init.HeUniform()`, since I am using ReLU layers.

In [ ]:
def scratch_net(input_var):
    
    net = {}
    net['data']  = layers.InputLayer(data_size, input_var=input_var)
    net['conv1'] = layers.Conv2DLayer(net['data'],  num_filters=4, filter_size=(1,7), pad='same', 
                                      nonlinearity=rectify)
    net['conv2'] = layers.Conv2DLayer(net['conv1'], num_filters=8, filter_size=(1,15), pad='same',
                                      stride=(1,2), nonlinearity=rectify)
    net['pool']  = layers.MaxPool2DLayer(net['conv2'], pool_size=(1,2))
    net['fcl']   = layers.DenseLayer(net['pool'], num_units=256, nonlinearity=rectify)
    net['out']   = layers.DenseLayer(net['fcl'], num_units=output_size, nonlinearity=sigmoid)
    
    return net

In [ ]:
def scratch_model(input_var, target_var, net):
    
    prediction = layers.get_output(net['out'])
    loss = binary_crossentropy(prediction, target_var)
    loss = lasagne.objectives.aggregate(loss)
    
    params = layers.get_all_params(net['out'], trainable=True)
    updates = lasagne.updates.rmsprop(loss, params, learning_rate=1e-5)
    
    test_prediction = layers.get_output(net['out'], deterministic=True)
    
    test_loss = binary_crossentropy(test_prediction, target_var)
    test_loss = lasagne.objectives.aggregate(test_loss)
    test_acc  = T.mean(lasagne.objectives.binary_accuracy(test_prediction, target_var),dtype=theano.config.floatX)
    
    train_fn = theano.function([input_var, target_var], loss, updates=updates)
    val_fn   = theano.function([input_var, target_var], [test_loss, test_acc])
    
    return train_fn, val_fn

In [ ]:
def scratch_train(train_fn, val_fn):
    train_err_list = []
    val_err_list = []
    val_acc_list = []
    
    print('| epoch \t| train loss\t| val loss\t| val acc\t| time')
    print('='*80)

    for epoch in range(num_epochs):
        st = time.time()
        batch_train_errs = []
        for batch in iterate_minibatches(x_train, y_train, batch_size):
            inputs, targets = batch
            err = train_fn(inputs, targets)
            batch_train_errs.append(err)   
        epoch_train_err = np.mean(batch_train_errs)
        train_err_list.append(epoch_train_err)
        
        batch_val_errs = []
        batch_val_accs = []
        for batch in iterate_minibatches(x_val, y_val, batch_size):
            inputs, targets = batch
            err, acc = val_fn(inputs, targets)
            batch_val_errs.append(err)
            batch_val_accs.append(acc)
        epoch_val_err = np.mean(batch_val_errs)
        val_err_list.append(epoch_val_err)
        epoch_val_acc = np.mean(batch_val_accs)
        val_acc_list.append(epoch_val_acc)
        
        en = time.time()
        print('| %d \t\t| %.6f\t| %.6f\t| %.2f%%\t| %.2f s' 
              % (epoch+1, epoch_train_err, epoch_val_err, epoch_val_acc * 100, en-st))
        #print('-'*80)
    return train_err_list, val_err_list, val_acc_list

In [ ]:
def scratch_test(val_fn):
    
    print('Test Results:')
    print('='*80)

    batch_err = []
    batch_acc = []
    for batch in iterate_minibatches(x_test, y_test, batch_size):
        inputs, targets = batch
        err, acc = val_fn(inputs, targets)
        batch_err.append(err)           
        batch_acc.append(acc)
            
    test_err = np.mean(batch_err)
    test_acc = np.mean(batch_acc)
    
    print('Test loss: %.6f \t| Test accuracy: %.2f' % (test_err, test_acc * 100))
    return test_err, test_acc

In [ ]:
num_epochs=100
batch_size=10

input_var = T.tensor4('inputs')
target_var = T.ivector('targets')
net = scratch_net(input_var)
train_fn, val_fn = scratch_model(input_var, target_var, net)

train_err, val_err, val_acc = scratch_train(train_fn, val_fn)

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(range(100), train_err, label='Training Error')
ax1.plot(range(100), val_err, label='Validation Error')

ax2 = ax1.twinx()
ax2.plot(range(100), np.asarray(val_acc) * 100, color='g', label='Validation accuracy')


----

## ConvNet Based On [This Site](https://github.com/luizgh/lasagne_basics/blob/master/comparing-optimization-algs.ipynb)

In [ ]:
data_size=(None,1,23,256) # Batch size x Img Channels x Height (electrodes) x Width (samples)
output_size=1             # Binary classification of seizure (1) vs non-seizure (0)

def build_model():
    input_var = T.tensor4('inputs')
    target_var = T.ivector('targets')
    net = {}

    #Input layer:
    net['data'] = lasagne.layers.InputLayer(data_size, input_var=input_var)

    #Convolution + Pooling
    net['conv1'] = lasagne.layers.Conv2DLayer(net['data'], num_filters=6, filter_size=(1,16), stride=(1,2), nonlinearity=rectify)
    net['pool1'] = lasagne.layers.Pool2DLayer(net['conv1'], pool_size=(1,2))

    net['conv2'] = lasagne.layers.Conv2DLayer(net['pool1'], num_filters=10, filter_size=(1,32), stride=(1,2), nonlinearity=rectify)
    net['pool2'] = lasagne.layers.Pool2DLayer(net['conv2'], pool_size=(1,2))


    #Fully-connected + dropout
    net['fc1'] = lasagne.layers.DenseLayer(net['pool2'], num_units=100)

    #Output layer:
    net['out'] = lasagne.layers.DenseLayer(net['fc1'], num_units=output_size, 
                                           nonlinearity=lasagne.nonlinearities.softmax)

    weight_decay = 1e-5

    #Loss function: mean cross-entropy
    prediction = lasagne.layers.get_output(net['out'])
    loss = lasagne.objectives.binary_crossentropy(prediction, target_var)
    loss = loss.mean()

    #Also add weight decay to the cost function
    weightsl2 = lasagne.regularization.regularize_network_params(net['out'], lasagne.regularization.l2)
    loss += weight_decay * weightsl2

    #Get the update rule
    params = lasagne.layers.get_all_params(net['out'], trainable=True)
    updates = lasagne.updates.sgd(loss, params, learning_rate=1e-2)

    test_prediction = lasagne.layers.get_output(net['out'], deterministic=True)
    test_loss = lasagne.objectives.binary_crossentropy(test_prediction,
                                                            target_var)
    test_loss = test_loss.mean()
    test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                      dtype=theano.config.floatX)

    train_fn = theano.function([input_var, target_var], loss, updates=updates, name='train')
    val_fn = theano.function([input_var, target_var], [test_loss, test_acc], name='validation')
    get_preds = theano.function([input_var], test_prediction, name='get_preds')

    return (train_fn, val_fn, get_preds)

In [ ]:
epochs = 10
batch_size=10

#Run the training function per mini-batches
n_examples = x_train.shape[0]
n_batches = int(n_examples / batch_size)

def train(train_fn):
    cost_history = []
    for epoch in range(epochs):
        st = time.time()
        batch_cost_history = []
        for batch in range(n_batches):
            x_batch = x_train[batch*batch_size: (batch+1) * batch_size]
            y_batch = y_train[batch*batch_size: (batch+1) * batch_size]

            this_cost = train_fn(x_batch, y_batch) # This is where the model gets updated

            batch_cost_history.append(this_cost)
        epoch_cost = np.mean(batch_cost_history)
        cost_history.append(epoch_cost)
        en = time.time()
        print('Epoch %d/%d, train error: %f. Elapsed time: %.2f seconds' % (epoch+1, epochs, epoch_cost, en-st))
    return cost_history

In [ ]:
sgd_functions = build_model()
for key, value in network.items():
    print('%s: %s' % (key, layers.get_output_shape(value)))
print ("Training with SGD")
sgd_cost_history = train(sgd_functions[0])

-----

## ConvNet Based On mnist.py 

In [ ]:
# ##################### Build the neural network model #######################
def build_cnn(input_var=None):
    data_size = (None,1,23,256)
    output_size = 1

    network = {}
    
    network['data'] = layers.InputLayer(shape=data_size, input_var=input_var)

    network['conv1'] = layers.Conv2DLayer(network['data'], num_filters=8, filter_size=(1, 15),
                                        stride=(1,1), nonlinearity=rectify,pad='same')

    network['pool1'] = layers.MaxPool2DLayer(network['conv1'], pool_size=(1,2))

    network['conv2'] = layers.Conv2DLayer(network['pool1'], num_filters=8, filter_size=(1, 15), 
                                        stride=(1,2), nonlinearity=rectify)
    
    network['dense1'] = layers.DenseLayer(network['conv2'],num_units=256, nonlinearity=rectify)

    network['out'] = layers.DenseLayer(network['dense1'], num_units=output_size, nonlinearity=rectify)

    return network

In [ ]:
# ############################# Batch iterator ###############################
# This is just a simple helper function iterating over training data in
# mini-batches of a particular size, optionally in random order. It assumes
# data is available as numpy arrays. For big datasets, you could load numpy
# arrays as memory-mapped files (np.load(..., mmap_mode='r')), or write your
# own custom data iteration function. For small datasets, you can also copy
# them to GPU at once for slightly improved performance. This would involve
# several changes in the main program, though, and is not demonstrated here.
# Notice that this function returns only mini-batches of size `batchsize`.
# If the size of the data is not a multiple of `batchsize`, it will not
# return the last (remaining) mini-batch.

def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [ ]:
# Prepare Theano variables for inputs and targets
input_var1 = T.tensor4('inputs')
target_var1 = T.ivector('targets')

In [ ]:
# Create neural network model (depending on first command line parameter)
print("Building model and compiling functions...")
network = build_cnn(input_var1)

In [ ]:
# Create a loss expression for training, i.e., a scalar objective we want
# to minimize (for our 2-class problem, it is the cross-entropy loss):
prediction1 = layers.get_output(network['out'])

loss1 = binary_crossentropy(prediction1, target_var1)
loss1 = loss1.mean()

# Update expressions for training
params1 = layers.get_all_params(network['out'], trainable=True)
updates1 = lasagne.updates.rmsprop(loss1, params1, learning_rate=0.001)

# Create a loss expression for validation/testing.
test_prediction1 = layers.get_output(network['out'], deterministic=True)

test_loss1 = binary_crossentropy(test_prediction1,target_var1)
test_loss1 = test_loss1.mean()

test_acc1 = T.mean(T.eq(T.argmax(test_prediction1, axis=1), target_var1),dtype=theano.config.floatX)

# Compile a function performing a training step on a mini-batch (by giving
# the updates dictionary) and returning the corresponding training loss:
train_fn1 = theano.function([input_var1, target_var1], loss1, updates=updates1)

# Compile a second function computing the validation loss and accuracy:
val_fn1 = theano.function([input_var1, target_var1], [test_loss1, test_acc1])

In [ ]:
for key, value in network.items():
    print('%s: %s' % (key, layers.get_output_shape(value)))

In [ ]:
num_epochs=10

# Finally, launch the training loop.
print("Starting training...")
# We iterate over epochs:
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(x_train, y_train, 10):
        inputs, targets = batch
        #print(inputs.shape, targets.shape)
        train_err += train_fn1(inputs, targets)
        train_batches += 1

    # And a full pass over the validation data:
    val_err = 0
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(x_val, y_val, 10):
        inputs, targets = batch
        err, acc = val_fn1(inputs, targets)
        val_err += err
        val_acc += acc
        val_batches += 1

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))
    print("  training error:\t\t{:.6f}".format(train_err)) #
    print("  train batches:\t\t{:d}".format(train_batches)) #
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
    print("  validation error:\t\t{:.6f}".format(val_err)) #
    print("  validation batches:\t\t{:d}".format(val_batches)) #
    print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))

In [ ]:
# After training, we compute and print the test error:
test_err = 0
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(x_test, y_test, 2, shuffle=False):
    inputs, targets = batch
    err, acc = val_fn(inputs, targets)
    test_err += err
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test loss:\t\t\t{:.6f}".format(test_err / test_batches))
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))